In [ ]:
!pip install bert-for-tf2

In [ ]:
import pandas as pd
import numpy as np
import bert
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import  Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tqdm import tqdm
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize.toktok import ToktokTokenizer
import re,string,unicodedata

In [ ]:
df=pd.read_csv('/data/review.csv')

In [ ]:
df.head(5)

In [ ]:
print("Nombre de colonnes et rangs : {}".format(df.shape))

In [ ]:
#Tokenization
tokenizer=ToktokTokenizer()
#Stopwords en anglais
stopword_list=nltk.corpus.stopwords.words('english')

In [ ]:
import re
from bs4 import BeautifulSoup
from nltk.tokenize import ToktokTokenizer
from nltk.corpus import stopwords

def clean_text(text):
    """suprrimer les htmls et les [] et les contenus dedans"""
    try:
        soup = BeautifulSoup(text, "html.parser")
        text = soup.get_text()
        
        text = re.sub('\[.*?\]', '', text)
    except Exception as e:
        print(f"Error cleaning text: {e}")
        text = ""
    
    return text

# appliquer
df['review'] = df['review'].apply(clean_text)

#enlever des caractères spéciales
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text

#appliquer sur le colonne de commentaire
df['review']=df['review'].apply(remove_special_characters)

#Lemmatisation
def stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text

#Apply function on review column
df['review']=df['review'].apply(stemmer)

#Enlever stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stop]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stop]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

# appliquer
df['review'] = df['review'].apply(remove_stopwords)

In [ ]:
# importer BERT et Tokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1", trainable=True)

# construire Tokenizer
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = bert.bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

# préparer les datas
input_data = convert_sentences_to_features(df['review'], tokenizer)

# construire le modèle
input_word_ids = Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name="input_word_ids")
input_mask = Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name="input_mask")
input_type_ids = Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name="input_type_ids")

bert_inputs = [input_word_ids, input_mask, input_type_ids]
bert_output = bert_layer(bert_inputs)[1]

dense = Dense(256, activation='relu')(bert_output)
dropout = Dropout(0.5)(dense)
output = Dense(1, activation='sigmoid')(dropout)  #2 types

model = Model(inputs=bert_inputs, outputs=output)
model.compile(Adam(lr=2e-5), loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


In [ ]:
tokenizer = create_tonkenizer(model.layers[3])

#utiliser les chiffres binaire
df['sentiment'].replace('positive',1.,inplace=True)
df['sentiment'].replace('negative',0.,inplace=True)

#mélanger le train et les ensembles de test
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], shuffle = True, test_size=0.20)

X_train = convert_sentences_to_features(X_train, tokenizer)
X_test = convert_sentences_to_features(X_test, tokenizer)


y_train = to_categorical(y_train)
y_test =  to_categorical(y_test)


In [ ]:
# Entraîner le modèle
BATCH_SIZE = 8
EPOCHS = 1

# Adam optimizer pour minimiser la perte categorical_crossentropy
opt = Adam(learning_rate=2e-5)
model.compile(optimizer=opt, 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Ajuster les données au modèle
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    verbose = 1)

# Enregistrer le modèle entraîné
model.save('nlp_model.h5') 

In [ ]:
# importer le modèle entraîné
from tensorflow.keras.models import load_model
new_model = load_model('nlp_model.h5',custom_objects={'KerasLayer':hub.KerasLayer})

In [ ]:
# Predire sur test-set
from sklearn.metrics import classification_report
pred_test = np.argmax(new_model.predict(X_test), axis=1)

In [ ]:
print(classification_report(np.argmax(y_test,axis=1), pred_test))